In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from measure_optimize import MeasurementOptimizer
import matplotlib.pyplot as plt
from itertools import permutations, product

## Input Setting

In [2]:
jaco_info = pd.read_csv('./RotaryBed/Q22_scale.csv', index_col=False)

jaco_list = np.asarray(jaco_info)

jaco = []
for i in range(len(jaco_list)):
    jaco.append(list(jaco_list[i][1:]))
    
print(np.shape(jaco))

(308, 5)


In [3]:
number_measure = 14
number_t = 22
number_total = number_measure*number_t
cost = [1]*number_total

## FIM collection and Sigma INV

In [4]:
solution1 = [1]*number_total

In [5]:
calculator = MeasurementOptimizer(jaco, number_measure, number_t, cost, verbose=True)

fim_expect = calculator.fim_computation()

calculator.compute_FIM(solution1)

Q shape: 308 5
======FIM result======
FIM: [[ 1.12672079e+03  1.99620709e+00 -8.27109148e+02  2.23307383e+03
  -1.27981839e+02]
 [ 1.99620709e+00  1.18589056e+00 -2.94047895e-01  1.63272938e+01
  -3.30259408e+00]
 [-8.27109148e+02 -2.94047895e-01  6.68554142e+02 -1.72703074e+03
   3.26205209e+01]
 [ 2.23307383e+03  1.63272938e+01 -1.72703074e+03  5.20782946e+03
  -2.12490410e+02]
 [-1.27981839e+02 -3.30259408e+00  3.26205209e+01 -2.12490410e+02
   1.16854209e+02]]
Determinant: 1435753590.9147868 ; log_e(det): 21.084955698778014 ; log_10(det): 9.157079911153815
Trace: 7121.144491083985 ; log_e(trace): 8.870823734618002 ; log_10(trace): 3.852549797880995
Min eig: 0.8358678444970318 ; log_e(min_eig): -0.17928475913869069 ; log_10(min_eig): -0.07786238158328697
Cond: 8134.8284627660305


array([[ 1.12672079e+03,  1.99620709e+00, -8.27109148e+02,
         2.23307383e+03, -1.27981839e+02],
       [ 1.99620709e+00,  1.18589056e+00, -2.94047895e-01,
         1.63272938e+01, -3.30259408e+00],
       [-8.27109148e+02, -2.94047895e-01,  6.68554142e+02,
        -1.72703074e+03,  3.26205209e+01],
       [ 2.23307383e+03,  1.63272938e+01, -1.72703074e+03,
         5.20782946e+03, -2.12490410e+02],
       [-1.27981839e+02, -3.30259408e+00,  3.26205209e+01,
        -2.12490410e+02,  1.16854209e+02]])

In [7]:
def sgn(p):
    """
    Give the signature of a permutation

    Parameters:
    -----------
    p: the permutation (a list)

    Return:
    ------
    1 if the number of exchange is an even number
    -1 if the number is an odd number
    """

    if len(p) == 1:
        return 1

    trans = 0

    for i in range(0, len(p)):
        j = i + 1

        for j in range(j, len(p)):
            if p[i] > p[j]:
                trans = trans + 1

    if (trans % 2) == 0:
        return 1
    else:
        return -1

In [8]:
def compute_FIM(numRes, FIMSet, obj="A", fix=False, budget=100):
    
    m = pyo.ConcreteModel()
    
    m.NumRes = pyo.Set(initialize=range(numRes))
    m.DimFIM = pyo.Set(initialize=range(len(FIMSet[0])))
    print(m.DimFIM)
    
    # set up y and cov y 
    #m.y = pyo.Var(m.NumRes, initialize=0.1, bounds=(0,1), within = pyo.NonNegativeReals)
    def identity(m, a, b):
        return 1 if a==b else 0

    m.cov_y = pyo.Var(m.NumRes, m.NumRes, initialize=identity, bounds=(0,1), within=pyo.NonNegativeReals)
    
    if fix:
        m.cov_y.fix()
    
    # set up FIM
    m.TotalFIM = pyo.Var(m.DimFIM, m.DimFIM, initialize=1)

    # compute FIM
    def eval_fim(m, a, b):
        return m.TotalFIM[a,b] == sum(m.cov_y[i,j]*FIMSet[i*numRes+j][a][b] for i in range(numRes) for j in range(numRes))
    
    
    # y and conv y constraints
    def y_covy1(m, a, b):
        return m.cov_y[a, b] <= m.cov_y[a, a]
    
    def y_covy2(m, a, b):
        return m.cov_y[a, b] <= m.cov_y[b, b]
    
    def y_covy3(m, a, b):
        return m.cov_y[a, b] >= m.cov_y[a, a] + m.cov_y[b, b] - 1
    
    def y_covy4(m, a, b):
        '''
        Avoid duplication
        '''
        if a>b:
            return m.cov_y[a, b] == m.cov_y[b, a]
        
        else:
            return pyo.Constraint.Skip
    
    
    
    def cost_compute(m):
        return m.cost == sum(m.cov_y[i,i] for i in range(numRes))
    
    def cost_limit(m):
        return m.cost <= budget
    
    # set up Design criterion
    def ComputeTrace(m):
        sum_x = sum(m.TotalFIM[j,j] for j in m.DimFIM)
        return sum_x
    
    
    def ComputeDet(m):
        """Calculate determinant. Can be applied to FIM of any size.
        det(A) = sum_{\sigma \in \S_n} (sgn(\sigma) * \Prod_{i=1}^n a_{i,\sigma_i})
        Use permutation() to get permutations, sgn() to get signature
        """

        r_list = list(range(len(m.DimFIM)))
        # get all permutations
        object_p = permutations(r_list)
        list_p = list(object_p)

        # generate a name_order to iterate \sigma_i
        det_perm = 0
        for i in range(len(list_p)):
            name_order = []
            x_order = list_p[i]
            # sigma_i is the value in the i-th position after the reordering \sigma
            for x in range(len(x_order)):
                for y, element in enumerate(m.DimFIM):
                    if x_order[x] == y:
                        name_order.append(element)

            det_perm += sgn(list_p[i]) * (m.TotalFIM[0,name_order[0]] *m.TotalFIM[1,name_order[1]]*m.TotalFIM[2,name_order[2]]*m.TotalFIM[3,name_order[3]]*m.TotalFIM[4,name_order[4]])

        return det_perm
    
    
    # add constraints
    m.TotalFIM_con = pyo.Constraint(m.DimFIM, m.DimFIM, rule=eval_fim)
    
    if not fix:
        m.cov1 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy1)
        m.cov2 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy2)
        m.cov3 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy3)
        m.cov4 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy4)
        
        m.cost = pyo.Var(initialize=budget)
        m.cost_compute = pyo.Constraint(rule=cost_compute)
        m.budget_limit = pyo.Constraint(rule=cost_limit)
    
    # set objective 
    if obj == "A":
        m.Obj = pyo.Objective(rule=ComputeTrace, sense=pyo.maximize)    
    elif obj =="D":
        m.Obj = pyo.Objective(rule=ComputeDet, sense=pyo.maximize)
    
    return m 

In [ ]:
def compute_FIM(numRes, FIMSet, obj="A", fix=False, budget=100):
    
    m = pyo.ConcreteModel()
    
    m.NumRes = pyo.Set(initialize=range(numRes))
    m.DimFIM = pyo.Set(initialize=range(len(FIMSet[0])))
    print(m.DimFIM)
    
    # set up y and cov y 
    #m.y = pyo.Var(m.NumRes, initialize=0.1, bounds=(0,1), within = pyo.NonNegativeReals)
    def identity(m, a, b):
        return 1 if a==b else 0

    m.cov_y = pyo.Var(m.NumRes, m.NumRes, initialize=identity, bounds=(0,1), within=pyo.NonNegativeReals)
    
    if fix:
        m.cov_y.fix()
    
    # set up FIM
    m.TotalFIM = pyo.Var(m.DimFIM, m.DimFIM, initialize=1)

    # compute FIM
    def eval_fim(m, a, b):
        return m.TotalFIM[a,b] == sum(m.cov_y[i,j]*FIMSet[i*numRes+j][a][b] for i in range(numRes) for j in range(numRes))
    
    
    # y and conv y constraints
    def y_covy1(m, a, b):
        return m.cov_y[a, b] <= m.cov_y[a, a]
    
    def y_covy2(m, a, b):
        return m.cov_y[a, b] <= m.cov_y[b, b]
    
    def y_covy3(m, a, b):
        return m.cov_y[a, b] >= m.cov_y[a, a] + m.cov_y[b, b] - 1
    
    def y_covy4(m, a, b):
        '''
        Avoid duplication
        '''
        if a>b:
            return m.cov_y[a, b] == m.cov_y[b, a]
        
        else:
            return pyo.Constraint.Skip
    
    
    
    def cost_compute(m):
        return m.cost == sum(m.cov_y[i,i] for i in range(numRes))
    
    def cost_limit(m):
        return m.cost <= budget
    
    # set up Design criterion
    def ComputeTrace(m):
        sum_x = sum(m.TotalFIM[j,j] for j in m.DimFIM)
        return sum_x
    
    
    def ComputeDet(m):
        """Calculate determinant. Can be applied to FIM of any size.
        det(A) = sum_{\sigma \in \S_n} (sgn(\sigma) * \Prod_{i=1}^n a_{i,\sigma_i})
        Use permutation() to get permutations, sgn() to get signature
        """

        r_list = list(range(len(m.DimFIM)))
        # get all permutations
        object_p = permutations(r_list)
        list_p = list(object_p)

        # generate a name_order to iterate \sigma_i
        det_perm = 0
        for i in range(len(list_p)):
            name_order = []
            x_order = list_p[i]
            # sigma_i is the value in the i-th position after the reordering \sigma
            for x in range(len(x_order)):
                for y, element in enumerate(m.DimFIM):
                    if x_order[x] == y:
                        name_order.append(element)

            det_perm += sgn(list_p[i]) * (m.TotalFIM[0,name_order[0]] *m.TotalFIM[1,name_order[1]]*m.TotalFIM[2,name_order[2]]*m.TotalFIM[3,name_order[3]]*m.TotalFIM[4,name_order[4]])

        return det_perm
    
    
    # add constraints
    m.TotalFIM_con = pyo.Constraint(m.DimFIM, m.DimFIM, rule=eval_fim)
    
    if not fix:
        m.cov1 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy1)
        m.cov2 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy2)
        m.cov3 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy3)
        m.cov4 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy4)
        
        m.cost = pyo.Var(initialize=budget)
        m.cost_compute = pyo.Constraint(rule=cost_compute)
        m.budget_limit = pyo.Constraint(rule=cost_limit)
    
    # set objective 
    if obj == "A":
        m.Obj = pyo.Objective(rule=ComputeTrace, sense=pyo.maximize)    
    elif obj =="D":
        m.Obj = pyo.Objective(rule=ComputeDet, sense=pyo.maximize)
    
    return m 

In [9]:
total_mea = 14*22

In [ ]:
mod = compute_FIM(total_mea, calculator.fim_collection, obj="D", fix=False, budget=310)

solver = pyo.SolverFactory('ipopt')
solver.options['linear_solver'] = 'ma57'
solver.solve(mod, tee=True)

DimFIM
Ipopt 3.13.2: linear_solver=ma57


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
      

  64 -4.5233053e+08 9.09e-13 3.46e+06  -1.0 3.68e+02   3.5 3.98e-03 4.89e-03f  1
  65 -4.5269620e+08 1.14e-12 3.46e+06  -1.0 5.00e+01   3.9 4.10e-02 1.51e-03f  1
Reallocating memory for MA57: lfact (28116673)
  66 -4.6051693e+08 3.64e-12 3.48e+06  -1.0 4.19e+02   3.5 3.81e-03 4.25e-03f  1
Reallocating memory for MA57: lfact (31786133)
  67 -4.6192559e+08 1.82e-12 3.46e+06  -1.0 5.58e+01   3.9 3.81e-02 5.63e-03f  1
  68 -4.6504586e+08 1.82e-12 3.46e+06  -1.0 4.32e+02   3.4 5.55e-03 1.85e-03f  1
  69 -4.6772367e+08 1.36e-12 3.43e+06  -1.0 6.18e+01   3.8 3.97e-02 1.07e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  70 -4.7494410e+08 4.55e-13 3.45e+06  -1.0 4.67e+02   3.4 1.01e-02 4.34e-03f  1
Reallocating memory for MA57: lfact (33747918)
  71 -4.7629395e+08 1.82e-12 3.43e+06  -1.0 6.69e+01   3.8 1.38e-01 5.62e-03f  1
  72 -4.8645199e+08 9.09e-13 3.47e+06  -1.0 4.60e+02   3.3 1.24e-02 6.97e-03f  1
  73 -4.8815823e+08 1.36e-12 3.44e+06  -1.0 7.19e

 152 -6.6323172e+08 3.64e-12 2.87e+06  -1.0 1.98e+03   2.2 1.04e-03 7.21e-04f  1
 153 -6.6367487e+08 2.27e-12 2.87e+06  -1.0 5.83e+04   1.7 1.63e-05 2.07e-05f  1
 154 -6.6435232e+08 3.18e-12 2.87e+06  -1.0 2.35e+03   2.1 7.57e-04 7.83e-04f  1


In [ ]:
print(pyo.value(mod.Obj))

In [ ]:
### FIM
fim_result = np.zeros((5,5))
for i in range(5):
    for j in range(5):
        fim_result[i,j] = pyo.value(mod.TotalFIM[i,j])
        
print(pyo.value(mod.TotalFIM[0,0]))
        
print(fim_result)
print('trace:', np.trace(fim_result))
print('det:', np.linalg.det(fim_result))

In [13]:
ans_y = np.zeros((total_mea,total_mea))


#for i in range(70):
    #print('Measurement', i, ':', pyo.value(mod.cov_y[i,i]))
    
for i in range(total_mea):
    for j in range(i, total_mea):
        cov = pyo.value(mod.cov_y[i,j])
        ans_y[i,j] = cov 
        ans_y[j,i] = pyo.value(mod.cov_y[j,i])
        
        if abs(pyo.value(mod.cov_y[i,j]) - pyo.value(mod.cov_y[j,i])) > 0.01:
            print('Wrong symmetry:', i, j, pyo.value(mod.cov_y[i,j]), pyo.value(mod.cov_y[j,i]))
        
        if abs(cov- min(pyo.value(mod.cov_y[i,i]), pyo.value(mod.cov_y[j,j]))) > 0.01:
            print('Wrong computation for covariance y:', i, j, pyo.value(mod.cov_y[i,i]), pyo.value(mod.cov_y[j,j]), cov)
#print(pyo.value(mod.cov_y[5,8]))
    
print(pyo.value(mod.cost))

Wrong computation for covariance y: 0 1 1 1 0
Wrong computation for covariance y: 0 2 1 1 0
Wrong computation for covariance y: 0 3 1 1 0
Wrong computation for covariance y: 0 4 1 1 0
Wrong computation for covariance y: 0 5 1 1 0
Wrong computation for covariance y: 0 6 1 1 0
Wrong computation for covariance y: 0 7 1 1 0
Wrong computation for covariance y: 0 8 1 1 0
Wrong computation for covariance y: 0 9 1 1 0
Wrong computation for covariance y: 0 10 1 1 0
Wrong computation for covariance y: 0 11 1 1 0
Wrong computation for covariance y: 0 12 1 1 0
Wrong computation for covariance y: 0 13 1 1 0
Wrong computation for covariance y: 0 14 1 1 0
Wrong computation for covariance y: 0 15 1 1 0
Wrong computation for covariance y: 0 16 1 1 0
Wrong computation for covariance y: 0 17 1 1 0
Wrong computation for covariance y: 0 18 1 1 0
Wrong computation for covariance y: 0 19 1 1 0
Wrong computation for covariance y: 0 20 1 1 0
Wrong computation for covariance y: 0 21 1 1 0
Wrong computation for 

Wrong computation for covariance y: 0 282 1 1 0
Wrong computation for covariance y: 0 283 1 1 0
Wrong computation for covariance y: 0 284 1 1 0
Wrong computation for covariance y: 0 285 1 1 0
Wrong computation for covariance y: 0 286 1 1 0
Wrong computation for covariance y: 0 287 1 1 0
Wrong computation for covariance y: 0 288 1 1 0
Wrong computation for covariance y: 0 289 1 1 0
Wrong computation for covariance y: 0 290 1 1 0
Wrong computation for covariance y: 0 291 1 1 0
Wrong computation for covariance y: 0 292 1 1 0
Wrong computation for covariance y: 0 293 1 1 0
Wrong computation for covariance y: 0 294 1 1 0
Wrong computation for covariance y: 0 295 1 1 0
Wrong computation for covariance y: 0 296 1 1 0
Wrong computation for covariance y: 0 297 1 1 0
Wrong computation for covariance y: 0 298 1 1 0
Wrong computation for covariance y: 0 299 1 1 0
Wrong computation for covariance y: 0 300 1 1 0
Wrong computation for covariance y: 0 301 1 1 0
Wrong computation for covariance y: 0 30

Wrong computation for covariance y: 2 72 1 1 0
Wrong computation for covariance y: 2 73 1 1 0
Wrong computation for covariance y: 2 74 1 1 0
Wrong computation for covariance y: 2 75 1 1 0
Wrong computation for covariance y: 2 76 1 1 0
Wrong computation for covariance y: 2 77 1 1 0
Wrong computation for covariance y: 2 78 1 1 0
Wrong computation for covariance y: 2 79 1 1 0
Wrong computation for covariance y: 2 80 1 1 0
Wrong computation for covariance y: 2 81 1 1 0
Wrong computation for covariance y: 2 82 1 1 0
Wrong computation for covariance y: 2 83 1 1 0
Wrong computation for covariance y: 2 84 1 1 0
Wrong computation for covariance y: 2 85 1 1 0
Wrong computation for covariance y: 2 86 1 1 0
Wrong computation for covariance y: 2 87 1 1 0
Wrong computation for covariance y: 2 88 1 1 0
Wrong computation for covariance y: 2 89 1 1 0
Wrong computation for covariance y: 2 90 1 1 0
Wrong computation for covariance y: 2 91 1 1 0
Wrong computation for covariance y: 2 92 1 1 0
Wrong computa

Wrong computation for covariance y: 3 185 1 1 0
Wrong computation for covariance y: 3 186 1 1 0
Wrong computation for covariance y: 3 187 1 1 0
Wrong computation for covariance y: 3 188 1 1 0
Wrong computation for covariance y: 3 189 1 1 0
Wrong computation for covariance y: 3 190 1 1 0
Wrong computation for covariance y: 3 191 1 1 0
Wrong computation for covariance y: 3 192 1 1 0
Wrong computation for covariance y: 3 193 1 1 0
Wrong computation for covariance y: 3 194 1 1 0
Wrong computation for covariance y: 3 195 1 1 0
Wrong computation for covariance y: 3 196 1 1 0
Wrong computation for covariance y: 3 197 1 1 0
Wrong computation for covariance y: 3 198 1 1 0
Wrong computation for covariance y: 3 199 1 1 0
Wrong computation for covariance y: 3 200 1 1 0
Wrong computation for covariance y: 3 201 1 1 0
Wrong computation for covariance y: 3 202 1 1 0
Wrong computation for covariance y: 3 203 1 1 0
Wrong computation for covariance y: 3 204 1 1 0
Wrong computation for covariance y: 3 20

Wrong computation for covariance y: 4 197 1 1 0
Wrong computation for covariance y: 4 198 1 1 0
Wrong computation for covariance y: 4 199 1 1 0
Wrong computation for covariance y: 4 200 1 1 0
Wrong computation for covariance y: 4 201 1 1 0
Wrong computation for covariance y: 4 202 1 1 0
Wrong computation for covariance y: 4 203 1 1 0
Wrong computation for covariance y: 4 204 1 1 0
Wrong computation for covariance y: 4 205 1 1 0
Wrong computation for covariance y: 4 206 1 1 0
Wrong computation for covariance y: 4 207 1 1 0
Wrong computation for covariance y: 4 208 1 1 0
Wrong computation for covariance y: 4 209 1 1 0
Wrong computation for covariance y: 4 210 1 1 0
Wrong computation for covariance y: 4 211 1 1 0
Wrong computation for covariance y: 4 212 1 1 0
Wrong computation for covariance y: 4 213 1 1 0
Wrong computation for covariance y: 4 214 1 1 0
Wrong computation for covariance y: 4 215 1 1 0
Wrong computation for covariance y: 4 216 1 1 0
Wrong computation for covariance y: 4 21

...

Wrong computation for covariance y: 255 293 1 1 0
Wrong computation for covariance y: 255 294 1 1 0
Wrong computation for covariance y: 255 295 1 1 0
Wrong computation for covariance y: 255 296 1 1 0
Wrong computation for covariance y: 255 297 1 1 0
Wrong computation for covariance y: 255 298 1 1 0
Wrong computation for covariance y: 255 299 1 1 0
Wrong computation for covariance y: 255 300 1 1 0
Wrong computation for covariance y: 255 301 1 1 0
Wrong computation for covariance y: 255 302 1 1 0
Wrong computation for covariance y: 255 303 1 1 0
Wrong computation for covariance y: 255 304 1 1 0
Wrong computation for covariance y: 255 305 1 1 0
Wrong computation for covariance y: 255 306 1 1 0
Wrong computation for covariance y: 255 307 1 1 0
Wrong computation for covariance y: 256 257 1 1 0
Wrong computation for covariance y: 256 258 1 1 0
Wrong computation for covariance y: 256 259 1 1 0
Wrong computation for covariance y: 256 260 1 1 0
Wrong computation for covariance y: 256 261 1 1 0


Wrong computation for covariance y: 264 272 1 1 0
Wrong computation for covariance y: 264 273 1 1 0
Wrong computation for covariance y: 264 274 1 1 0
Wrong computation for covariance y: 264 275 1 1 0
Wrong computation for covariance y: 264 276 1 1 0
Wrong computation for covariance y: 264 277 1 1 0
Wrong computation for covariance y: 264 278 1 1 0
Wrong computation for covariance y: 264 279 1 1 0
Wrong computation for covariance y: 264 280 1 1 0
Wrong computation for covariance y: 264 281 1 1 0
Wrong computation for covariance y: 264 282 1 1 0
Wrong computation for covariance y: 264 283 1 1 0
Wrong computation for covariance y: 264 284 1 1 0
Wrong computation for covariance y: 264 285 1 1 0
Wrong computation for covariance y: 264 286 1 1 0
Wrong computation for covariance y: 264 287 1 1 0
Wrong computation for covariance y: 264 288 1 1 0
Wrong computation for covariance y: 264 289 1 1 0
Wrong computation for covariance y: 264 290 1 1 0
Wrong computation for covariance y: 264 291 1 1 0


Wrong computation for covariance y: 273 298 1 1 0
Wrong computation for covariance y: 273 299 1 1 0
Wrong computation for covariance y: 273 300 1 1 0
Wrong computation for covariance y: 273 301 1 1 0
Wrong computation for covariance y: 273 302 1 1 0
Wrong computation for covariance y: 273 303 1 1 0
Wrong computation for covariance y: 273 304 1 1 0
Wrong computation for covariance y: 273 305 1 1 0
Wrong computation for covariance y: 273 306 1 1 0
Wrong computation for covariance y: 273 307 1 1 0
Wrong computation for covariance y: 274 275 1 1 0
Wrong computation for covariance y: 274 276 1 1 0
Wrong computation for covariance y: 274 277 1 1 0
Wrong computation for covariance y: 274 278 1 1 0
Wrong computation for covariance y: 274 279 1 1 0
Wrong computation for covariance y: 274 280 1 1 0
Wrong computation for covariance y: 274 281 1 1 0
Wrong computation for covariance y: 274 282 1 1 0
Wrong computation for covariance y: 274 283 1 1 0
Wrong computation for covariance y: 274 284 1 1 0


Wrong computation for covariance y: 285 292 1 1 0
Wrong computation for covariance y: 285 293 1 1 0
Wrong computation for covariance y: 285 294 1 1 0
Wrong computation for covariance y: 285 295 1 1 0
Wrong computation for covariance y: 285 296 1 1 0
Wrong computation for covariance y: 285 297 1 1 0
Wrong computation for covariance y: 285 298 1 1 0
Wrong computation for covariance y: 285 299 1 1 0
Wrong computation for covariance y: 285 300 1 1 0
Wrong computation for covariance y: 285 301 1 1 0
Wrong computation for covariance y: 285 302 1 1 0
Wrong computation for covariance y: 285 303 1 1 0
Wrong computation for covariance y: 285 304 1 1 0
Wrong computation for covariance y: 285 305 1 1 0
Wrong computation for covariance y: 285 306 1 1 0
Wrong computation for covariance y: 285 307 1 1 0
Wrong computation for covariance y: 286 287 1 1 0
Wrong computation for covariance y: 286 288 1 1 0
Wrong computation for covariance y: 286 289 1 1 0
Wrong computation for covariance y: 286 290 1 1 0


AttributeError: 'ConcreteModel' object has no attribute 'cost'

In [ ]:
print(ans_y)

figure = plt.figure()
axes = figure.add_subplot(111)

caxes = axes.matshow(ans_y, interpolation='nearest')
figure.colorbar(caxes)

plt.show()

In [ ]:
sol_y = np.asarray([ans_y[i,i] for i in range(total_mea)])

sol_y = np.reshape(sol_y, (14,5))
np.around(sol_y)
print(sol_y)

In [ ]:
import matplotlib

def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw=None, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (M, N).
    row_labels
        A list or array of length M with the labels for the rows.
    col_labels
        A list or array of length N with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if ax is None:
        ax = plt.gca()

    if cbar_kw is None:
        cbar_kw = {}

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(np.arange(data.shape[1]), labels=col_labels)
    ax.set_yticks(np.arange(data.shape[0]), labels=row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)
        
        
    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [ ]:
measure_name = ['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']

time_set = ['500', '1000', '1500', '2000', '2200']
#time_set = np.linspace(100, 2200, 22)
time_set = [str(i) for i in time_set]

fig, ax = plt.subplots(figsize=(16,8))

im, cbar = heatmap(sol_y, measure_name, time_set, ax=ax,
                   cmap="YlGn", cbarlabel="weight")
texts = annotate_heatmap(im, valfmt="{x:.1f}")

fig.tight_layout()
plt.show()

In [ ]:
def plot_data(budget, a_cri, d_cri, design_cri_label):
    sense = []

    #plt.scatter(budget, np.log10(a_cri), s=50, color='red', label='A-optimality')
    plt.scatter(budget, np.log10(d_cri), marker='v', s=50, color='blue', label='D-optimality')
    plt.xlabel('Budget')
    plt.ylabel('log10(' + design_cri_label + ')')
    plt.legend()

    plt.show()

In [ ]:
a_set = [869.53, 1496.64, 1538.93, 1563.76, 1567.70, 1569.73, 1571.23]
d_set = [140.57, 2852.58, 11349.32, 23168.51, 30715.88, 34988.78, 36029.47]

plot_data([10,20,30,40,50,60,70], a_set, d_set, 'Determinant')